In [2]:
import pandas as pd

df_a = pd.read_csv('data/TopArtists.csv')
df_t = pd.read_csv('data/TopTracks.csv')
df_g = pd.read_csv('data/TopGenres.csv')
df_m = pd.read_csv('data/MusicFeatures.csv')

In [3]:
df_a['user_id'].unique()

array([12153521253, 12120382831])

In [11]:
u1_a = df_a.loc[df_a['user_id'] == 12153521253]
u1_t = df_t.loc[df_t['user_id'] == 12153521253]
u1_g = df_g.loc[df_g['user_id'] == 12153521253]
u1_m = df_m.loc[df_m['user_id'] == 12153521253]

In [12]:
u2_a = df_a.loc[df_a['user_id'] == 12120382831]
u2_t = df_t.loc[df_t['user_id'] == 12120382831]
u2_g = df_g.loc[df_g['user_id'] == 12120382831]
u2_m = df_m.loc[df_m['user_id'] == 12120382831]

## Artists

In [13]:
u1_a.head()

,user_id,rank,artist_id,timeframe,artist,genres,artist_url,artist_image,popularity
0,12153521253,1,3OpWOvjJnk86geCHrAxkHA,Long,Sungha Jung,acoustic guitar cover; fingerstyle; viral pop,https://open.spotify.com/artist/3OpWOvjJnk86ge...,https://i.scdn.co/image/2084aff5eff3416d8fa89d...,50
3,12153521253,2,6eUKZXaKkcviH0Ku9w2n3V,Long,Ed Sheeran,pop; uk pop,https://open.spotify.com/artist/6eUKZXaKkcviH0...,https://i.scdn.co/image/c9e693f336bc004af00c51...,92
5,12153521253,3,04gDigrS5kc9YWfZHwBETP,Long,Maroon 5,pop; pop rock,https://open.spotify.com/artist/04gDigrS5kc9YW...,https://i.scdn.co/image/305c7c493ca5c108ade08d...,88
6,12153521253,4,44insiIQApkRaCMIbuaISJ,Long,Joey Pecoraro,chillhop; lo-fi beats,https://open.spotify.com/artist/44insiIQApkRaC...,https://i.scdn.co/image/af76a3b950e634842d96ad...,60
9,12153521253,5,7n2wHs1TKAczGzO7Dd2rGr,Long,Shawn Mendes,canadian pop; dance pop; pop; post-teen pop; v...,https://open.spotify.com/artist/7n2wHs1TKAczGz...,https://i.scdn.co/image/809b93c47fca237138e539...,92


In [14]:
u2_a.head()

,user_id,rank,artist_id,timeframe,artist,genres,artist_url,artist_image,popularity
1,12120382831,1,5K4W6rqBFWDnAN6FQUkS6x,Long,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8a...,90
2,12120382831,2,2elBjNSdBE2Y3f0j1mjrql,Long,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,https://i.scdn.co/image/0e5952203ea0140aa5687f...,73
4,12120382831,3,73sIBHcqh3Z3NyqHKZ7FOL,Long,Childish Gambino,atl hip hop; hip hop; pop rap; rap,https://open.spotify.com/artist/73sIBHcqh3Z3Ny...,https://i.scdn.co/image/2e1d974c14fb66eb5050bf...,80
7,12120382831,4,1Xyo4u8uXC1ZmMpatF05PJ,Long,The Weeknd,canadian contemporary r&b; canadian pop; pop,https://open.spotify.com/artist/1Xyo4u8uXC1ZmM...,https://i.scdn.co/image/d9a875c37277c35b94c60c...,94
8,12120382831,5,50co4Is1HCEo8bhOyUWKpn,Long,Young Thug,atl hip hop; atl trap; gangster rap; melodic r...,https://open.spotify.com/artist/50co4Is1HCEo8b...,https://i.scdn.co/image/69ef089edd03d6d5455ed6...,90


In [33]:
u1_a['popularity'].mean()

76.8135593220339

In [20]:
a1 = u1_a.loc[u1_a['timeframe'] == 'Long']
a2 = u2_a.loc[u2_a['timeframe'] == 'Long']

In [21]:
a1['artist_id'].unique()

array(['3OpWOvjJnk86geCHrAxkHA', '6eUKZXaKkcviH0Ku9w2n3V',
       '04gDigrS5kc9YWfZHwBETP', '44insiIQApkRaCMIbuaISJ',
       '7n2wHs1TKAczGzO7Dd2rGr', '5K4W6rqBFWDnAN6FQUkS6x',
       '3AQRLZ9PuTAozP28Skbq8V', '2YZyLoL8N0Wb9xBt1NhZWg',
       '2h93pZq0e7k5yf4dywlkpM', '20wkVLutqVOYrc0kxFs7rA',
       '4jGPdu95icCKVF31CcFKbS', '3TVXtAsR1Inumwj472S9r4',
       '5pKCCKE2ajJHZ9KAiaK11H', '5JZ7CnR6gTvEMKX4g70Amv',
       '1Bl6wpkWCQ4KVgnASpvzzA', '3WGpXCj9YhhfX11TToZcXP',
       '6qqNVTkY8uBg9cP3Jd7DAH', '49tQo2QULno7gxHutgccqF',
       '3mIj9lX2MWuHmhNCA7LSCW', '1anyVhU62p31KFi8MEzkbf',
       '3KV3p5EY4AvKxOlhGHORLg', '1uNFoZAHBGtllmzznpCI3s',
       '1QAJqy2dA3ihHBFIHRphZj', '4AK6F7OLvEQ5QYCBNiQWHq',
       '2wY79sveU1sp5g7SokKOiI', '1Xyo4u8uXC1ZmMpatF05PJ',
       '69GGBxA162lTqCwzJG5jLp', '5nCi3BB41mBaMH9gfr6Su0',
       '6N1GBr0koYYU0W59ECgg5u', '5P5FTygHyx2G57oszR3Wot',
       '7CQwac16i1W5ej8YpuL3dv', '73sIBHcqh3Z3NyqHKZ7FOL',
       '4gzpq5DPGxSnKTe4SA8HAU', '4BxCuXFJrSWGi1KHcVqaU4

In [22]:
a2['artist_id'].unique()

array(['5K4W6rqBFWDnAN6FQUkS6x', '2elBjNSdBE2Y3f0j1mjrql',
       '73sIBHcqh3Z3NyqHKZ7FOL', '1Xyo4u8uXC1ZmMpatF05PJ',
       '50co4Is1HCEo8bhOyUWKpn', '69GGBxA162lTqCwzJG5jLp',
       '3TVXtAsR1Inumwj472S9r4', '45eNHdiiabvmbp4erw26rg',
       '13ubrt8QOOCPljQ2FL1Kca', '1Bl6wpkWCQ4KVgnASpvzzA',
       '4V8LLVI7PbaPR0K2TGSxFF', '1EowJ1WwkMzkCkRomFhui7',
       '2YZyLoL8N0Wb9xBt1NhZWg', '68kEuyFKyqrdQQLLsmiatm',
       '3MZsBdqDrRTJihTHQrO6Dq', '00FQb4jTyendYWaN8pK0wa',
       '5IcR3N7QB1j6KBL8eImZ8m', '7CajNmpbOovFoOoasH2HaY',
       '23fqKkggKUBHNkbKtXEls4', '3mIj9lX2MWuHmhNCA7LSCW',
       '2h93pZq0e7k5yf4dywlkpM', '7dGJo4pcD2V6oG8kP0tJRR',
       '3jK9MiCrA42lLAdMGUZpwa', '0Y5tJX1MQlPlqiwlOH1tJY',
       '0fA0VVWsXO9YnASrzqfmYu', '7iZtZyCzp3LItcw1wtPI3D',
       '6l3HvQ5sa6mXTsMTB19rO5', '1vCWHaC5f2uS3yhpwWbIA6',
       '4O15NlyKLIASxsJ0PrXPfz', '6oMuImdp5ZcFhWP0ESe6mG',
       '1QAJqy2dA3ihHBFIHRphZj', '1VPmR4DJC1PlOtd0IADAO0',
       '2ZRQcIgzPCVaT9XKhXZIzh', '7aA592KWirLsnfb5ulGWvU